In [19]:
import os
import random
import torch

# 指定路径
folder_path = r'D:\Ultra-low-field-MRI-Scanner-main\hku__EMI\data\all_data'

# 获取文件夹中所有 .pth 文件
pth_files = [f for f in os.listdir(folder_path) if f.endswith('.pth')]

# 随机选择一个 .pth 文件并构建路径
random_pth_file = random.choice(pth_files)
file_path = os.path.join(folder_path, random_pth_file)

# 加载 .pth 文件中的字典
data_dict = torch.load(file_path)

# 提取 'k-space' 和 'label' 数据
Kspace_data = data_dict['k-space']
label_data = data_dict['label']


#随机读取和按顺序读取

In [20]:
import torch

def shift_data_on_height(data, new_size=50):
    """
    对数据的行维度（height, 即128维度）进行平移操作，分别向前平移2行、1行、不平移、向后平移1行、2行，
    并将结果组合到列维度，得到形状为 [batch, channel, height, new_size] 的张量。
    
    data: 输入数据，形状为 [batch, channel, height, width]
    new_size: 目标列数，默认为50
    """
    batch, channel, height, width = data.shape
    assert width * 5 == new_size, "目标 new_size 应该是 width 的5倍"  # 确保 new_size 是列维度的5倍
    
    # 创建一个空的张量，用于保存五种操作的结果，目标形状为 [batch, channel, height, new_size]
    new_data = torch.zeros(batch, channel, height, new_size)
    
    # 向前平移2行
    shift_f2 = torch.zeros_like(data)
    shift_f2[..., :-2, :] = data[..., 2:, :]  # 顶部补0
    # 向前平移1行
    shift_f1 = torch.zeros_like(data)
    shift_f1[..., :-1, :] = data[..., 1:, :]  # 顶部补0
    
    # 不平移
    shift_0 = data.clone()

    # 向后平移1行
    shift_b1 = torch.zeros_like(data)
    shift_b1[..., 1:, :] = data[..., :-1, :]  # 底部补0

    # 向后平移2行
    shift_b2 = torch.zeros_like(data)
    shift_b2[..., 2:, :] = data[..., :-2, :]  # 底部补0

    """"
    ### 将五种操作结果沿列维度（width）拼接在一起
    new_data[..., :10] = shift_f2
    new_data[..., 10:20] = shift_f1
    new_data[..., 20:30] = shift_0
    new_data[..., 30:40] = shift_b1
    new_data[..., 40:50] = shift_b2
    """
    new_data[..., ::5] = shift_f2  # 将 shift_f2 填入列索引为 0, 5, 10, 15, 20, 25, 30, 35, 40, 45
    new_data[..., 1::5] = shift_f1  # 将 shift_f1 填入列索引为 1, 6, 11, 16, 21, 26, 31, 36, 41, 46
    new_data[..., 2::5] = shift_0   # 将 shift_0 填入列索引为 2, 7, 12, 17, 22, 27, 32, 37, 42, 47
    new_data[..., 3::5] = shift_b1  # 将 shift_b1 填入列索引为 3, 8, 13, 18, 23, 28, 33, 38, 43, 48
    new_data[..., 4::5] = shift_b2  # 将 shift_b2 填入列索引为 4, 9, 14, 19, 24, 29, 34, 39, 44, 49

    return new_data

# 假设原始数据的 shape 是 [1, 2, 128, 10]
Kspace_data = torch.randn(1, 2, 128, 10)

# 调用函数，生成目标形状为 [1, 2, 128, 50] 的数据
Kspace_shifted = shift_data_on_height(Kspace_data)

print(f"Original K-space shape: {Kspace_data.shape}")
print(f"Shifted K-space shape: {Kspace_shifted.shape}")


Original K-space shape: torch.Size([1, 2, 128, 10])
Shifted K-space shape: torch.Size([1, 2, 128, 50])


In [21]:
import os
import torch
"""
# 假设 shift_data_on_height 函数已经定义
def shift_data_on_height(data):
    # 示例的平移函数，按行维度（高度）平移数据
    shift_f2 = torch.zeros_like(data)
    shift_f1 = torch.zeros_like(data)
    shift_0 = torch.zeros_like(data)
    shift_b1 = torch.zeros_like(data)
    shift_b2 = torch.zeros_like(data)

    shift_f2[..., :-2, :] = data[..., 2:, :]
    shift_f1[..., :-1, :] = data[..., 1:, :]
    shift_0[..., :, :] = data
    shift_b1[..., 1:, :] = data[..., :-1, :]
    shift_b2[..., 2:, :] = data[..., :-2, :]

    new_data = torch.zeros(data.shape[:-1] + (50,))
    new_data[..., ::5] = shift_f2
    new_data[..., 1::5] = shift_f1
    new_data[..., 2::5] = shift_0
    new_data[..., 3::5] = shift_b1
    new_data[..., 4::5] = shift_b2

    return new_data
"""
# 函数实现
def process_pth_files(set_type):
    # 设置路径
    folder_path = os.path.join(r'D:\Ultra-low-field-MRI-Scanner-main\hku__EMI\data', set_type)
    
    # 检查对应的文件夹是否存在
    if not os.path.exists(folder_path):
        print(f"{set_type} 文件夹不存在: {folder_path}")
        return
    
    # 获取所有 .pth 文件
    pth_files = [f for f in os.listdir(folder_path) if f.endswith('.pth')]
    
    # 如果没有 .pth 文件，返回提示
    if not pth_files:
        print(f"{set_type} 文件夹中没有 .pth 文件")
        return
    
    # 遍历文件夹中的每个 .pth 文件
    for pth_file in pth_files:
        file_path = os.path.join(folder_path, pth_file)
        
        # 加载 .pth 文件
        data_dict = torch.load(file_path)
        
        # 提取 k-space 和 label 数据
        Kspace_data = data_dict.get('k-space')
        label_data = data_dict.get('label')
        
        # 处理 Kspace_data
        Kspace_shifted = shift_data_on_height(Kspace_data)
        
        # 设置新的保存文件夹路径为 'D:\Ultra-low-field-MRI-Scanner-main\hku__EMI\shifted\{set_type}'
        shifted_folder = os.path.join(r'D:\Ultra-low-field-MRI-Scanner-main\hku__EMI\shifted', set_type)
        if not os.path.exists(shifted_folder):
            os.makedirs(shifted_folder)  # 创建文件夹
        
        # 保存的路径和文件名
        save_path = os.path.join(shifted_folder, pth_file)
        
        # 将处理后的数据保存为 .pth 文件
        torch.save({'k-space': Kspace_shifted, 'label': label_data}, save_path)
        print(f"处理后的数据已保存到: {save_path}")

# 示例调用函数：分别处理 test、train、validation 文件夹
process_pth_files('test')
process_pth_files('train')
process_pth_files('validation')

处理后的数据已保存到: D:\Ultra-low-field-MRI-Scanner-main\hku__EMI\shifted\test\0.pth
处理后的数据已保存到: D:\Ultra-low-field-MRI-Scanner-main\hku__EMI\shifted\test\1.pth
处理后的数据已保存到: D:\Ultra-low-field-MRI-Scanner-main\hku__EMI\shifted\test\10.pth
处理后的数据已保存到: D:\Ultra-low-field-MRI-Scanner-main\hku__EMI\shifted\test\100.pth
处理后的数据已保存到: D:\Ultra-low-field-MRI-Scanner-main\hku__EMI\shifted\test\1000.pth
处理后的数据已保存到: D:\Ultra-low-field-MRI-Scanner-main\hku__EMI\shifted\test\1001.pth
处理后的数据已保存到: D:\Ultra-low-field-MRI-Scanner-main\hku__EMI\shifted\test\1002.pth
处理后的数据已保存到: D:\Ultra-low-field-MRI-Scanner-main\hku__EMI\shifted\test\1003.pth
处理后的数据已保存到: D:\Ultra-low-field-MRI-Scanner-main\hku__EMI\shifted\test\1004.pth
处理后的数据已保存到: D:\Ultra-low-field-MRI-Scanner-main\hku__EMI\shifted\test\1005.pth
处理后的数据已保存到: D:\Ultra-low-field-MRI-Scanner-main\hku__EMI\shifted\test\1006.pth
处理后的数据已保存到: D:\Ultra-low-field-MRI-Scanner-main\hku__EMI\shifted\test\1007.pth
处理后的数据已保存到: D:\Ultra-low-field-MRI-Scanner-main\hku__EMI\shif

In [16]:
import os

# 指定保存文件夹路径
save_folder = r'D:\Ultra-low-field-MRI-Scanner-main\hku__EMI\shifted\test'

# 使用 random_pth_file 的值作为文件名
save_file_name = random_pth_file  # 假设 random_pth_file 已经被定义

# 组合完整保存路径
save_path = os.path.join(save_folder, save_file_name)

# 创建新的字典
data_to_save = {
    'k-space': Kspace_shifted,  # 将 Kspace_shifted 对应的 key 改为 'k-space'
    'label': label_data         # label_data 对应的 key 为 'label'
}

# 保存字典为 .pth 文件
torch.save(data_to_save, save_path)

print(f"数据已保存到 {save_path}")


tensor([[[ 18.3848,  -0.6073,   3.5648, -20.6964,  -2.0690,   2.2885,   4.5788,
           -4.7734,  15.4956,  24.5181,  22.1898,  -0.1068,  10.1814,   3.2199,
            3.8594,   4.7174,   6.0396, -46.2398,  -2.3442,  -7.0212,   7.3096,
            3.4160,   3.7730, -12.7704,  12.9858,  -8.5307, -32.4741,   2.9492,
          -11.9577,  13.6375,  18.1314,  -2.8440, -17.2660,  -0.7804,  -7.6504,
           13.1580,  -4.8373,  20.8078,  17.3960,  11.1412,   3.2794,  -0.6099,
           -6.9085, -15.4734,  -9.1950,   5.4585,  -9.9195,   5.1093,  -8.7353,
            7.1985, -14.8822,   3.4258,  27.0593,  -9.7016, -26.4441,   5.1946,
            0.5249,  -3.9910,  -5.3044,   0.6471,   7.2043,  -3.0236,   1.1545,
          -22.0913,   2.1806,  15.3437,  -3.6955,   0.4618, -10.2484,   8.9089,
           26.3108,   1.8365,  -3.8517,   1.1192,   1.8995,   8.0965,   4.0208,
           -9.6473,   5.9642,  16.4188,   6.3089,   2.9846, -10.6244,  -5.5334,
            4.9624, -10.7069,  29.0349, 